In [1]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
import numpy as np

In [2]:
client=QdrantClient('http://localhost:6333')

In [3]:
EMBEDDING_MODEL='jinaai/jina-embeddings-v2-small-en'
query='I just discovered the course. Can I join now?'

### Q1. Embedding the query

In [17]:
embedding_model = TextEmbedding(model_name=EMBEDDING_MODEL)
query_embedding = np.array(list(embedding_model.embed(query))[0])
print(np.min(embedding1))

-0.11726373885183883


In [7]:
np.linalg.norm(embedding1)

np.float64(1.0)

In [9]:
q = np.array(embedding1)
q = q.flatten()
q.dot(q)

np.float64(1.0000000000000002)

### Q2. Cosine similarity with another vector

In [11]:
doc = 'Can I still join the course after the start date?'
embedding_model = TextEmbedding(model_name=EMBEDDING_MODEL)
embedding2 = np.array(list(embedding_model.embed([doc]))[0])

In [13]:
np.dot(embedding1,embedding2)

np.float64(0.9008528895674548)

### Q3. Ranking by cosine

In [14]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [15]:
docs = [doc['text'] for doc in documents]

In [16]:
doc_embeddings = np.array(list(embedding_model.embed(docs)))

In [19]:
cosine_similarities = doc_embeddings.dot(query_embedding)
max_similarity_index = np.argmax(cosine_similarities)
max_similarity_index

np.int64(1)

### Q4. Ranking by cosine, version two

In [23]:
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
texts_embedding = np.array(list(embedding_model.embed(full_texts)))
cosine_sim = texts_embedding.dot(query_embedding)
max_sim = np.argmax(cosine_sim)
max_sim

np.int64(0)

### Q5. Selecting the embedding model

In [25]:
dimensions = [model['dim'] for model in TextEmbedding.list_supported_models()]
min(dimensions)

384

In [26]:
model = 'BAAI/bge-small-en'

### Q6. Indexing with qdrant (2 points)


In [27]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [31]:
collection_name='zoomcamp-hw2'
EMBEDDING_DIMENSIONALITY=384
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [35]:
points=[]

for i, doc in enumerate(documents):
    text=doc['question']+' '+doc['text']
    vector=models.Document(text=text, model=model)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [36]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [46]:
result = client.query_points(
             collection_name=collection_name,
             query=models.Document(
                 text=query,
                 model=model
             ),
             limit=5,
             with_payload=True
         )
result.points[0].score

0.8703172